<a href="https://colab.research.google.com/github/neelmehta00/RealEstateMoguls/blob/master/Traditional_Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
def rec(text, i):
    locdate = text.find("Date:")
    loccomma = text.find(",",locdate)
    locprice = text.find("Price:")
    loccommap = text.find("(",locprice)
    price = (text[locprice+8: loccommap-1])
    finaldate = (text[locdate+6:loccomma])
    if(locdate == -1):
        return
    finalprice = ""
    for ch in price:
        if ch != ",":
            finalprice = finalprice + ch
    pricehist = [finaldate, finalprice]
    newtext = text[loccommap:]
    final[i].append(pricehist)
    rec(newtext, i)
def basics(houselink, i):

    with requests.Session() as s:
        url = houselink
        r = s.get(url, headers=req_headers)
    soup = BeautifulSoup(r.content, 'lxml')
    try:
        price = soup.find("span", {"class": "ds-value"})
    except:
        price = "0"
    try:
        bed = soup.findAll("span", {"class": "ds-bed-bath-living-area"}) #bed bath and sqft in order
    except:
        price = "0"
    try:
        address = soup.find("h1", {"class": "ds-address-container"})
    except:
        price = "0"
    try:
        year = soup.findAll("span", {"class": "ds-body ds-home-fact-value"})
    except:
        price = "0"
    try:
        finalyear = year[1].get_text()
    except:
        finalyear = "0000"
    #list for all the data of one house
    spechouse = []


    


    

    #Start of address
    try:
        address = address.get_text()
    except:
        address = "N/A"
    print(address)
    address = str(address)
    finalad = ""
    for ch in address:
        if ch != ",":
            finalad = finalad + ch
    #End of address

    #price of one house
    #start of price of one house code
    price = str(price)
    price = re.sub('[^0-9]' , '', price)
    #spechouse.append(price)
    #end of price of one house code
    #start of bed, bath, sqft
    bed = str(bed)
    bedlist = bed.split('>')

    templist = []
    for item in bedlist:
        inlist = []
        for st in item:
            if st.isdigit():
                inlist.append(st)
        templist.append(inlist)
    templist = ([x for x in templist if x])
    count = 0
    temper = []
    for item in templist:
        temper.append(item)
        count = count + 1
        if(count == 3):
            break
    foot = ""
    try:
        for num in temper[2]:
            foot = foot + num
    except:
        foot = "0000"
    try:
        spechouse = [houselink, finalad, price, temper[0][0], temper[1][0], foot, finalyear]
    except:
        spechouse = ["N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A"]
    #end of bed, bath, sqft
    #f.write(finalad + "," + price + "," + temper[0][0] + "," + temper[1][0] + "," + foot + "\n" )
    final[i].append(spechouse)
    return(spechouse)
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}
urllink = []
link1 = []
urllink.append("https://www.zillow.com/mount-pleasant-sc/luxury-homes//homes/for_sale/house,condo,apartment_duplex,mobile,townhouse_type/priced_sort/0_mmm/16_p/")
for link in urllink:
    with requests.Session() as s:
        url = link
        r = s.get(url, headers=req_headers)
    soup = BeautifulSoup(r.content, 'lxml')
    content = soup.findAll("a",{"class":"list-card-img"})
    content = str(content) 
    quotation = content.split('"')
    count = 0
    for item in quotation:  
        if (item.find("https://www.zillow.com/homedetail") != -1):
            link1.append(item)
#making the file start
filename = "housing16page.csv"
f = open(filename, "w")
headers = "link, Address, Price, #Bed, #Bath, Square Foot, Year Built, date1, price1, date2, price2, date3, price3, date4, price4, date5, price5, date6, price6\n"
f.write(headers)
#end of making the file
final = []
for item in link1:
    final.append([])
#end of basics function
#puts each sublist into the final list
finallist = []
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
#options.add_argument('--headless')
DRIVER_PATH = '/Users/ryuno/Downloads/chromedriver'
browser = webdriver.Chrome(options=options,executable_path=DRIVER_PATH)
for i in range(len(link1)):
    #finallist.append(basics(item)) #the argument in basics should be replaced with item
    basics(link1[i], i)
    browser.get(link1[i])
    time.sleep(6)
    html = browser.page_source
    soup = BeautifulSoup(html, "lxml")
    #start of cleaning up price history code
    table = soup.find("table", {"class": "oj51b6-2 hxGMPN"})
    try:
        pricehist = table.get_text()
    except:
        pricehist = "0"
    pricehist = str(pricehist)
    rec(pricehist, i)
    print(pricehist)
for i in range(len(final)):
    biglist = []
    for j in range(len(final[i])):
        for k in range(len(final[i][j])):
            biglist.append(final[i][j][k])
    for x in range(len(biglist)):
        f.write(biglist[x] + ",")
    f.write("\n")
f.close()